In [1]:
import pandas as pd
from Func_extr import *
from Func_viz import *
from Func_analisis  import *     
import sqlite3

In [13]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')

cursor = conn.cursor()

query = f'''
        SELECT location_id, latitude, longitude FROM locations_id WHERE Type = "Copernicus";
    '''

df_loc = pd.read_sql_query(query, conn)
# Ejecutar la consulta
cursor.execute(query)
conn.close()


In [46]:
pixels_corrPos  = []
pixels_corrNeg  = []
for pixel in df_loc['location_id']:
    df = create_df(pixel)
    if df.empty:
        pass
    else:
        variables = ['total_precipitation']
        df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
        df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
        df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
        df = df.sort_values('date').set_index('date')
        df = df.select_dtypes(exclude=['datetime64'])
        correlaciones = df.corr()
        prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]
        media_corr = correlaciones[prec_columns].T['quantity_hm3'].mean()
        if media_corr>0:
            pixels_corrPos.append(pixel)
        else:
            pixels_corrNeg.append(pixel)

In [50]:
df = create_df(189)
if df.empty:
    pass
else:
    variables = ['total_precipitation']
    df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
    df = df.sort_values('date').set_index('date')
    df = df.select_dtypes(exclude=['datetime64'])
    correlaciones = df.corr()
    prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]
    media_corr = correlaciones[prec_columns].T['quantity_hm3'].mean()

In [55]:
correlaciones[prec_columns].T['quantity_hm3'].sum()

0.22457019759294397

In [47]:
pixels_corrPos

[179,
 182,
 186,
 188,
 189,
 190,
 191,
 192,
 194,
 195,
 196,
 197,
 198,
 200,
 201,
 202,
 203,
 204,
 294,
 299]

In [48]:
pixels_corrNeg

[185]

In [56]:
df = create_df(185)
if df.empty:
    pass
else:
    variables = ['total_precipitation']
    df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
    df = df.sort_values('date').set_index('date')
    df = df.select_dtypes(exclude=['datetime64'])
    correlaciones = df.corr()
    prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]

In [59]:
 correlaciones[prec_columns].T['quantity_hm3']

total_precipitation                -0.042466
total_precipitation_mean_last1D    -0.042466
total_precipitation_mean_last7D    -0.058085
total_precipitation_mean_last15D   -0.032308
total_precipitation_mean_last20D   -0.000698
total_precipitation_mean_last1M    -0.026894
total_precipitation_mean_last2M    -0.023519
total_precipitation_mean_last3M    -0.000625
total_precipitation_mean_last4M     0.018848
total_precipitation_mean_last1Y    -0.384173
total_precipitation_mean_last2Y    -0.551763
total_precipitation_mean_last3Y          NaN
Name: quantity_hm3, dtype: float64